# Prepare

For preparations I will follow a fast aproach and I will remove all columns that are not helpfull.

**import**

In [1]:
# data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# data
import requests

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# spliting the data
from sklearn.model_selection import TimeSeriesSplit
from math import ceil

# system manipulation
import sys
sys.path.append("./util_")
import acquire_
# import prepare_

# other
import warnings
warnings.filterwarnings("ignore")

### get data

In [2]:
# get data from acquire file
dogs = acquire_.get_dogs_data()
dogs.head()

,DateOfBite,Species,Breed,Age,Gender,SpayNeuter,Borough,ZipCode
UniqueID,,,,,,,,
1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220
2,January 04 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,NaN
3,January 06 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224
4,January 08 2018,DOG,Mixed/Other,4,M,False,Brooklyn,11231
5,January 09 2018,DOG,Pit Bull,NaN,U,False,Brooklyn,11224


**Rename columns**

In [3]:
# make the columsn lower case
# replace spaces
dogs.columns = dogs.columns.str.lower().str.strip().str.replace("†","").str.replace(" ","_")
dogs.head(1)

,dateofbite,species,breed,age,gender,spayneuter,borough,zipcode
UniqueID,,,,,,,,
1,January 01 2018,DOG,UNKNOWN,NaN,U,False,Brooklyn,11220


**Convert data types**

In [4]:
# view datatypes
dogs.dtypes

dateofbite    object
species       object
breed         object
age           object
gender        object
spayneuter      bool
borough       object
zipcode       object
dtype: object

In [5]:
# count the unique value counts in each column
for i in dogs.columns:
    print("_" * 20, i.upper())
    print(dogs[i].value_counts().value_counts())

____________________ DATEOFBITE
7     299
9     276
6     267
8     228
10    219
5     208
11    192
12    164
4     127
13    124
14     97
3      93
15     65
16     52
17     38
2      37
1      16
19     15
18     14
20     11
21      4
22      4
24      2
25      2
23      1
Name: dateofbite, dtype: int64
____________________ SPECIES
22663    1
Name: species, dtype: int64
____________________ BREED
1      1186
2       162
3        54
4        39
5        24
       ... 
95        1
106       1
123       1
127       1
53        1
Name: breed, Length: 89, dtype: int64
____________________ AGE
1       122
2        31
3         8
4         7
6         6
9         6
5         4
20        3
10        2
13        2
11        2
7         2
50        2
45        1
8         1
14        1
17        1
22        1
26        1
27        1
33        1
34        1
1624      1
1504      1
242       1
1365      1
1178      1
1040      1
795       1
655       1
569       1
375       1
361       1
1

In [6]:
# convert string to datetime
dogs.dateofbite = dogs.dateofbite.astype("datetime64")

**Find nulls**

In [7]:
dogs.isna().sum()

dateofbite        0
species           0
breed          2218
age           11221
gender            0
spayneuter        0
borough           0
zipcode        5858
dtype: int64

**Drop duplicate**

In [8]:
# remove duplicated rows
dogs = dogs.drop_duplicates()

**Drop columns**

In [9]:
dogs.columns

Index(['dateofbite', 'species', 'breed', 'age', 'gender', 'spayneuter',
       'borough', 'zipcode'],
      dtype='object')

In [10]:
# extract date column and remove everything else
dogs = dogs[dogs.columns[dogs.columns.isin(["dateofbite"])]]
dogs.head()

,dateofbite
UniqueID,
1,2018-01-01
2,2018-01-04
3,2018-01-06
4,2018-01-08
5,2018-01-09


**Create my target**

In [11]:
# create my target bite column
dogs["bite"] = int(1)

In [12]:
# set the date to be the indexinf column
dogs = dogs.set_index("dateofbite").sort_index()
dogs.head(1)

,bite
dateofbite,
2015-01-01,1


**Frequency of time**

In [16]:
# occurrences of each date
dogs.index.value_counts()

2015-06-06    25
2017-09-16    25
2018-06-23    24
2018-07-04    23
2018-06-02    22
              ..
2020-05-08     1
2020-10-06     1
2021-03-02     1
2016-03-14     1
2021-12-31     1
Name: dateofbite, Length: 2555, dtype: int64

In [18]:
# occurrences of each dates occurrence
dogs.index.value_counts().value_counts()

6     289
7     286
9     275
8     248
10    227
5     224
11    196
12    147
13    126
4     126
3      99
14     88
15     56
16     39
2      38
17     34
1      16
19     12
18     11
20      8
21      3
22      3
25      2
24      1
23      1
Name: dateofbite, dtype: int64

**Check for gaps in time**

In [19]:
# earliest date
min_date = dogs.index.min()
min_date

Timestamp('2015-01-01 00:00:00')

In [20]:
# latest date
max_date = dogs.index.max()
max_date

Timestamp('2021-12-31 00:00:00')

In [13]:
# # find all year strings ages
# dogs[dogs.Age == ""]

In [14]:
# # get all the cells that are not nulls
# non_null_dogs = dogs[dogs.Age.notna()]

# # find all year strings ages
# y = non_null_dogs[non_null_dogs.Age.str.contains(r"[yY]", regex=True)]

# # use extract to only get the numbers in the string cells representing yesr
# y["good_year"] = y.Age.str.extract("(\d+)")
# y.head()

In [15]:
# # find all month strings ages
# m = non_null_dogs[non_null_dogs.Age.str.contains(r"[mM]", regex=True)]
# m["good_year"] = m.Age.str.extract("(\d+)").astype(int) / 12
# m.head()

In [16]:
# # find all weeks strings ages
# w = non_null_dogs[non_null_dogs.Age.str.contains(r"[KkwW]", regex=True)]
# w["good_year"] = w.Age.str.extract("(\d+)").astype(int) / 54
# w.head()
